In [1]:
from aiida import load_profile

load_profile()

Profile<uuid='c677ac2b24bc40178ad9f1681fc23d0a' name='presto'>

In [2]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

#structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

In [3]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [4]:
from aiida.orm import load_code
code = load_code("janus@localhost")

In [5]:
from aiida.orm import Float, Dict, Str

inputs = {
    "code": code,
    "model": model,
    "struct": structure,
    "arch": Str(model.architecture),
    "device": Str("cpu"),
    "calc_kwargs": Dict({}),
    "min_volume": Float(0.95),
    "max_volume": Float(1.05),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
}

In [6]:
from aiida.plugins import CalculationFactory
eosCalc = CalculationFactory("mlip.eos")

In [7]:
from aiida.engine import run_get_node
result, node = run_get_node(eosCalc, inputs)

11/24/2025 08:20:57 PM <215324> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
11/24/2025 08:20:57 PM <215324> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
11/24/2025 08:20:57 PM <215324> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.


outputs <AttributesFrozendict {'code': <InstalledCode: Remote code 'janus' on localhost pk: 1, uuid: 294ec8d5-4250-4245-8040-d20377fa8233>, 'model': <ModelData: uuid: 8fb3d4e2-5d7a-4ea1-9b41-d821624e0712 (pk: 2)>, 'struct': <StructureData: uuid: 6b552a48-9e49-464c-bdac-4d20cbdad460 (pk: 517)>, 'arch': <Str: uuid: bbf3c541-4e14-4fc0-8030-063e758108bd (pk: 518) value: mace_mp>, 'device': <Str: uuid: 901389cd-9a93-41d8-946e-021cf67d2c4b (pk: 519) value: cpu>, 'calc_kwargs': <Dict: uuid: 28e11169-d744-45c5-ba8f-2b1aa190ce01 (pk: 520)>, 'min_volume': <Float: uuid: 26504f04-85e1-4f38-979c-0a317528e2d1 (pk: 521) value: 0.95>, 'max_volume': <Float: uuid: eb5f596f-f5c8-40fe-a451-beb62afa7472 (pk: 522) value: 1.05>, 'metadata': <AttributesFrozendict {'options': <AttributesFrozendict {'resources': {'num_machines': 1}, 'input_filename': 'aiida.xyz', 'output_filename': 'aiida-outputs.xyz', 'submit_script_filename': '_aiidasubmit.sh', 'scheduler_stdout': '_scheduler-stdout.txt', 'scheduler_stderr': 

In [ ]:
print(result)
print(node)

In [ ]:
from aiida.orm import load_node

folder = load_node(340)

print(folder.list_object_names())
print(folder.get_object_content('aiida-outputs.xyz'))